<a href="https://colab.research.google.com/github/ayachaieb/kaggle-solutions/blob/gh-pages/model_ppp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Model Architecture *

In [16]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [17]:
from google.colab import drive
drive.mount('/content/drive')
path= '/content/drive/MyDrive/dataset-delay.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
df = pd.read_csv(path, encoding_errors='ignore')
df.shape

(1000, 10)

In [19]:
df['Delay'] = abs(pd.to_datetime(df[' delivary date']) - pd.to_datetime(df['demand date']))
df['Delay'] = df['Delay'].dt.days
df.to_csv('/content/drive/MyDrive/dataset.csv',index=False)

<ipython-input-19-9ffde1a97902>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Delay'] = abs(pd.to_datetime(df[' delivary date']) - pd.to_datetime(df['demand date']))
<ipython-input-19-9ffde1a97902>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Delay'] = abs(pd.to_datetime(df[' delivary date']) - pd.to_datetime(df['demand date']))


In [25]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['supplier state'] = encoder.fit_transform(df['supplier state'])
df['ordermethod'] = encoder.fit_transform(df['ordermethod'])
df['supplier'] = encoder.fit_transform(df['supplier'])
df['material'] = encoder.fit_transform(df['material'])

In [27]:
from sklearn.model_selection import train_test_split
X = df[['supplier-id', 'order-id', 'order quantity','supplier','material' , 'order date', 'supplier state', 'ordermethod']]
y = df['Delay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [29]:
import numpy as np

timesteps = 5  # Number of past orders to use as input
n_features = 8

def create_dataset(X, y, look_back=timesteps):
    X_data = []
    y_data = []
    for i in range(len(X) - look_back):
        a = X[i:(i + look_back), :]
        X_data.append(a)
        y_data.append(y[i + look_back])
    return np.array(X_data), np.array(y_data)

X_train, y_train = create_dataset(X_train.values, y_train.values)
X_test, y_test = create_dataset(X_test.values, y_test.values)

In [44]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define model parameters
timesteps = 20  # Length of the input sequence
num_features = 8  # Number of features in your input data
num_classes = 3  # Number of possible decision classes

# Input layer
input_layer = keras.Input(shape=(timesteps, num_features))

# LSTM layer
lstm_layer = layers.LSTM(units=128, return_sequences=True)(input_layer)

# Dense layers (shared for both outputs)
shared_dense = layers.Dense(64, activation='relu')(lstm_layer)

# Output layers
delay_output = layers.Dense(1, activation='linear', name='delay')(shared_dense)
decision_output = layers.Dense(num_classes, activation='softmax', name='decision')(shared_dense)

# Define the model
model = keras.Model(inputs=input_layer, outputs=[delay_output, decision_output])



In [47]:
def categorize_decision(delay):
    if delay < 3:
        return "Regular Process"
    elif 3 <= delay <= 7:
        return "Expedite Shipment"
    else:
        return "Negotiate with Supplier"

df['Decision'] = df['Delay'].apply(categorize_decision)
encoder = LabelEncoder()
df['Decision'] = encoder.fit_transform(df['Decision'])

# 3. Split Decision Data
y_train_decision = df['Decision'][:len(X_train)]
y_val_decision = df['Decision'][len(X_train):]
y_train_decision = y_train_decision.astype('float32')
y_val_decision = y_val_decision.astype('float32')



In [59]:
def convert_date(date_string):
    date_format = "%b %d%s %y"
    try:
        if date_string[-2:] == 'st':
            return datetime.datetime.strptime(date_string, date_format)
        elif date_string[-2:] == 'nd':
            return datetime.datetime.strptime(date_string, date_format)
        elif date_string[-2:] == 'rd':
            return datetime.datetime.strptime(date_string, date_format)
        elif date_string[-2:] == 'th':
            return datetime.datetime.strptime(date_string, date_format)
        else:
            raise ValueError("Invalid date format")
    except ValueError as e:
        print(f"Error converting date: {date_string}, Error: {e}")
        return None  # Or handle the invalid date as needed

In [60]:
import datetime
df['demand date'] = df['demand date'].apply(convert_date)
df['order date'] = df['order date'].apply(convert_date)
df[' delivary date'] = df[' delivary date'].apply(convert_date)

Error converting date: Dec 1st 23, Error: Invalid date format
Error converting date: Oct 6th 22, Error: Invalid date format
Error converting date: Nov 13th 23, Error: Invalid date format
Error converting date: Feb 15th 24, Error: Invalid date format
Error converting date: Sep 30th 23, Error: Invalid date format
Error converting date: Jun 9th 23, Error: Invalid date format
Error converting date: Oct 11th 22, Error: Invalid date format
Error converting date: Aug 11th 22, Error: Invalid date format
Error converting date: Nov 3rd 22, Error: Invalid date format
Error converting date: Mar 16th 24, Error: Invalid date format
Error converting date: Feb 28th 24, Error: Invalid date format
Error converting date: Jan 23rd 24, Error: Invalid date format
Error converting date: Jul 7th 22, Error: Invalid date format
Error converting date: Feb 7th 24, Error: Invalid date format
Error converting date: Nov 2nd 23, Error: Invalid date format
Error converting date: Sep 9th 23, Error: Invalid date format


In [49]:


# Compile the model
model.compile(
    optimizer='adam',
    loss={'delay': 'mean_squared_error', 'decision': 'sparse_categorical_crossentropy'},  # Sparse for integer encoding
    metrics={'delay': 'mae', 'decision': 'accuracy'}
)

# Train the model
history = model.fit(
    X_train,
    {'delay': y_train, 'decision': y_train_decision},
    epochs=50,  # Adjust epochs as needed
    validation_data=(X_test, {'delay': y_test, 'decision': y_val_decision}),
)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
# Evaluate the model on the test set
X_test, y_test_delay, y_test_decision = ... # Your test data
results = model.evaluate(X_test, {'delay': y_test_delay, 'decision': y_test_decision})

# Print the evaluation results
print('Test Loss:', results[0])
print('Test Delay MSE:', results[1])
print('Test Decision Accuracy:', results[2])

# You can also make predictions using the trained model
predictions = model.predict(X_test)
predicted_delay = predictions[0]
predicted_decision = predictions[1]